## Simulated annealing

In [9]:
pip install scikit-optimize

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import scipy.stats as st
import timeit
from joblib import Parallel, delayed
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

## Simulated annealing with periodic review and MCS

In [7]:
start = timeit.default_timer()

def calculate_safety_stock(z, std_dev_daily_sales, lead_time):
    safety_stock = z * std_dev_daily_sales * np.sqrt(lead_time)
    return safety_stock

summary = {
    'Purchase Cost': [12, 7, 6, 37],
    'Lead Time': [9, 6, 15, 12],
    'Size': [0.57, 0.05, 0.53, 1.05],
    'Selling Price': [16.10, 8.60, 10.20, 68],
    'Starting Stock': [2750, 22500, 5200, 1400],
    'Ch': [0.2 * 12, 0.2 * 7, 0.2 * 6, 0.2 * 37],
    'Co': [1000, 1200, 1000, 1200],
    'Probability': [0.76, 1.00, 0.70, 0.23],
    'Mean Demand (Lead Time)': [103.50, 648.55, 201.68, 150.06],
    'Std. Dev. of Demand (Lead Time)': [37.32, 26.45, 31.08, 3.21],
    'Expected Demand (Lead Time)': [705, 3891, 2266, 785]
}

class Product:
    def __init__(self, i):
        self.mean = np.mean([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.sd = np.std([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.i = i
        self.unit_cost = summary['Purchase Cost'][i - 1]
        self.lead_time = summary['Lead Time'][i - 1]
        self.size = summary['Size'][i - 1]
        self.selling_price = summary['Selling Price'][i - 1]
        self.holding_cost = summary['Ch'][i - 1]
        self.ordering_cost = summary['Co'][i - 1]
        self.probability = summary['Probability'][i - 1]
        self.starting_stock = summary['Starting Stock'][i - 1]
        self.mean_demand_lead_time = summary['Mean Demand (Lead Time)'][i - 1]
        self.std_dev_demand_lead_time = summary['Std. Dev. of Demand (Lead Time)'][i - 1]
        self.expected_demand_lead_time = summary['Expected Demand (Lead Time)'][i - 1]

def daily_demand(mean, sd, probability):
    if np.random.uniform(0, 1) > probability:
        return 0
    else:
        return np.exp(np.random.normal(mean, sd))

def MCS_periodic(product, review_period=15, z_score=1.65, min_order_ratio=0.2):
    inventory = product.starting_stock
    mean = product.mean
    sd = product.sd
    lead_time = product.lead_time
    probability = product.probability
    demand_lead = product.expected_demand_lead_time

    high_demand_value = 2 * mean
    weights = [0.8, 0.2]
    means = [mean, high_demand_value]
    sds = [sd, sd]

    daily_sales = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
    std_dev_daily_sales = np.std(daily_sales)

    safety_stock = calculate_safety_stock(z_score, std_dev_daily_sales, lead_time)
    reorder_point = safety_stock + demand_lead

    min_order_quantity = max(int(min_order_ratio * mean), 1)

    q = 0
    stock_out = 0
    counter = 0
    order_placed = False
    data = {'inv_level': [], 'daily_demand': [], 'units_sold': [], 'units_lost': [], 'orders': [], 'reorder_quantities': []}

    total_safety_stock = 0

    for day in range(1, 365):
        day_demand = daily_demand(mean, sd, probability)
        data['daily_demand'].append(day_demand)

        if day % review_period == 0 and not order_placed:
            q = max(safety_stock + demand_lead - inventory, min_order_quantity)
            order_placed = True
            data['orders'].append(q)
            data['reorder_quantities'].append(q)

        if order_placed:
            counter += 1

        if counter == lead_time:
            inventory += q
            order_placed = False
            counter = 0

        if inventory - day_demand >= 0:
            data['units_sold'].append(day_demand)
            inventory -= day_demand
        else:
            data['units_sold'].append(inventory)
            data['units_lost'].append(day_demand - inventory)
            inventory = 0
            stock_out += 1

        data['inv_level'].append(inventory)
        total_safety_stock += safety_stock

    average_safety_stock = total_safety_stock / 365

    return data, average_safety_stock

def profit_calculation(data, product):
    unit_cost = product.unit_cost
    selling_price = product.selling_price
    holding_cost = product.holding_cost
    order_cost = product.ordering_cost
    size = product.size
    days = 365

    revenue = sum(data['units_sold']) * selling_price
    Co = len(data['orders']) * order_cost
    Ch = sum(data['inv_level']) * holding_cost * size / days
    cost = sum(data['orders']) * unit_cost

    profit = revenue - cost - Co - Ch

    return profit

def simulated_annealing(product, initial_temp, final_temp, alpha, iterations):
    current_temp = initial_temp
    current_solution = [15, 0.2]  # [review_period, min_order_ratio]
    best_solution = current_solution
    best_profit = -np.inf

    while current_temp > final_temp:
        for _ in range(iterations):
            # Generate a new solution by perturbing the current solution
            new_solution = [
                max(1, current_solution[0] + np.random.randint(-1, 2)),  # Perturb review period
                max(0.1, min(1.0, current_solution[1] + np.random.uniform(-0.05, 0.05)))  # Perturb min_order_ratio
            ]

            data, _ = MCS_periodic(product, review_period=new_solution[0], min_order_ratio=new_solution[1])
            new_profit = profit_calculation(data, product)

            # Accept the new solution with a probability
            if new_profit > best_profit or np.exp((new_profit - best_profit) / current_temp) > np.random.rand():
                current_solution = new_solution
                best_profit = new_profit
                best_solution = current_solution

        # Cool down the temperature
        current_temp *= alpha

    return best_solution, best_profit

products = [Product(i) for i in range(1, 5)]
results = {}
initial_temp = 1000
final_temp = 0.1
alpha = 0.9
iterations = 100

for product in products:
    best_solution, best_profit = simulated_annealing(product, initial_temp, final_temp, alpha, iterations)
    data, avg_safety_stock = MCS_periodic(product, review_period=best_solution[0], min_order_ratio=best_solution[1])
    profit = profit_calculation(data, product)
    lost_order_proportion = len(data['units_lost']) / 365
    reorder_quantity = np.mean(data['reorder_quantities'])

    results[f'Pr{product.i}'] = {
        'best_review_period': best_solution[0],
        'best_min_order_ratio': best_solution[1],
        'mean_profit': profit,
        'lost_order_proportion': lost_order_proportion,
        'reorder_quantity': reorder_quantity,
        'average_safety_stock': avg_safety_stock
    }

for key, value in results.items():
    print(f"Product {key}:")
    print(f"  Best Review Period: {value['best_review_period']}")
    print(f"  Best Min Order Ratio: {value['best_min_order_ratio']}")
    print(f"  Mean Profit: {value['mean_profit']}")
    print(f"  Proportion of Lost Orders: {value['lost_order_proportion']}")
    print(f"  Average Reorder Quantity: {value['reorder_quantity']}")
    print(f"  Average Safety Stock: {value['average_safety_stock']}\n")

stop = timeit.default_timer()
print('Time: ', stop - start)


Product Pr1:
  Best Review Period: 9
  Best Min Order Ratio: 0.14862529039282232
  Mean Profit: 92252.26292095124
  Proportion of Lost Orders: 0.6356164383561644
  Average Reorder Quantity: 605.9796589978507
  Average Safety Stock: 12.15689103470783

Product Pr2:
  Best Review Period: 6
  Best Min Order Ratio: 0.8277840414650315
  Mean Profit: 457205.329580156
  Proportion of Lost Orders: 0.8246575342465754
  Average Reorder Quantity: 3846.7320421675045
  Average Safety Stock: 12.511515770064367

Product Pr3:
  Best Review Period: 15
  Best Min Order Ratio: 0.218718331364789
  Mean Profit: 178240.2403551467
  Proportion of Lost Orders: 0.6219178082191781
  Average Reorder Quantity: 1709.8827060361666
  Average Safety Stock: 20.459205463916085

Product Pr4:
  Best Review Period: 16
  Best Min Order Ratio: 0.1
  Mean Profit: 433555.8428913356
  Proportion of Lost Orders: 0.2
  Average Reorder Quantity: 616.5235572692734
  Average Safety Stock: 15.3519856384228

Time:  85.45603739994112


## Simulated annealing with continuous review and MCS

In [8]:
start = timeit.default_timer()

def calculate_safety_stock(z, std_dev_daily_sales, lead_time):
    safety_stock = z * std_dev_daily_sales * np.sqrt(lead_time)
    return safety_stock

summary = {
    'Purchase Cost': [12, 7, 6, 37],
    'Lead Time': [9, 6, 15, 12],
    'Size': [0.57, 0.05, 0.53, 1.05],
    'Selling Price': [16.10, 8.60, 10.20, 68],
    'Starting Stock': [2750, 22500, 5200, 1400],
    'Ch': [0.2 * 12, 0.2 * 7, 0.2 * 6, 0.2 * 37],
    'Co': [1000, 1200, 1000, 1200],
    'Probability': [0.76, 1.00, 0.70, 0.23],
    'Mean Demand (Lead Time)': [103.50, 648.55, 201.68, 150.06],
    'Std. Dev. of Demand (Lead Time)': [37.32, 26.45, 31.08, 3.21],
    'Expected Demand (Lead Time)': [705, 3891, 2266, 785]
}

class Product:
    def __init__(self, i):
        self.mean = np.mean([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.sd = np.std([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.i = i
        self.unit_cost = summary['Purchase Cost'][i - 1]
        self.lead_time = summary['Lead Time'][i - 1]
        self.size = summary['Size'][i - 1]
        self.selling_price = summary['Selling Price'][i - 1]
        self.holding_cost = summary['Ch'][i - 1]
        self.ordering_cost = summary['Co'][i - 1]
        self.probability = summary['Probability'][i - 1]
        self.starting_stock = summary['Starting Stock'][i - 1]
        self.mean_demand_lead_time = summary['Mean Demand (Lead Time)'][i - 1]
        self.std_dev_demand_lead_time = summary['Std. Dev. of Demand (Lead Time)'][i - 1]
        self.expected_demand_lead_time = summary['Expected Demand (Lead Time)'][i - 1]

def daily_demand(mean, sd, probability):
    if np.random.uniform(0, 1) > probability:
        return 0
    else:
        return np.exp(np.random.normal(mean, sd))

def MCS_continuous(product, z_score=1.65, min_order_ratio=0.2):
    inventory = product.starting_stock
    mean = product.mean
    sd = product.sd
    lead_time = product.lead_time
    probability = product.probability
    demand_lead = product.expected_demand_lead_time

    high_demand_value = 2 * mean
    weights = [0.8, 0.2]
    means = [mean, high_demand_value]
    sds = [sd, sd]

    daily_sales = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
    std_dev_daily_sales = np.std(daily_sales)

    safety_stock = calculate_safety_stock(z_score, std_dev_daily_sales, lead_time)
    reorder_point = safety_stock + demand_lead

    min_order_quantity = max(int(min_order_ratio * mean), 1)

    q = 0
    stock_out = 0
    counter = 0
    order_placed = False
    data = {'inv_level': [], 'daily_demand': [], 'units_sold': [], 'units_lost': [], 'orders': [], 'reorder_quantities': []}

    total_safety_stock = 0

    for day in range(1, 365):
        day_demand = daily_demand(mean, sd, probability)
        data['daily_demand'].append(day_demand)

        if inventory <= reorder_point and not order_placed:
            q = max(reorder_point - inventory + min_order_quantity, min_order_quantity)
            order_placed = True
            data['orders'].append(q)
            data['reorder_quantities'].append(q)

        if order_placed:
            counter += 1

        if counter == lead_time:
            inventory += q
            order_placed = False
            counter = 0

        if inventory - day_demand >= 0:
            data['units_sold'].append(day_demand)
            inventory -= day_demand
        else:
            data['units_sold'].append(inventory)
            data['units_lost'].append(day_demand - inventory)
            inventory = 0
            stock_out += 1

        data['inv_level'].append(inventory)
        total_safety_stock += safety_stock

    average_safety_stock = total_safety_stock / 365

    return data, average_safety_stock

def profit_calculation(data, product):
    unit_cost = product.unit_cost
    selling_price = product.selling_price
    holding_cost = product.holding_cost
    order_cost = product.ordering_cost
    size = product.size
    days = 365

    revenue = sum(data['units_sold']) * selling_price
    Co = len(data['orders']) * order_cost
    Ch = sum(data['inv_level']) * holding_cost * size / days
    cost = sum(data['orders']) * unit_cost

    profit = revenue - cost - Co - Ch

    return profit

def simulated_annealing(product, initial_temp, final_temp, alpha, iterations):
    current_temp = initial_temp
    current_solution = [1.65, 0.2]  # [z_score, min_order_ratio]
    best_solution = current_solution
    best_profit = -np.inf

    while current_temp > final_temp:
        for _ in range(iterations):
            # Generate a new solution by perturbing the current solution
            new_solution = [
                max(1.0, min(3.0, current_solution[0] + np.random.uniform(-0.1, 0.1))),  # Perturb z_score
                max(0.1, min(1.0, current_solution[1] + np.random.uniform(-0.05, 0.05)))  # Perturb min_order_ratio
            ]

            data, _ = MCS_continuous(product, z_score=new_solution[0], min_order_ratio=new_solution[1])
            new_profit = profit_calculation(data, product)

            # Accept the new solution with a probability
            if new_profit > best_profit or np.exp((new_profit - best_profit) / current_temp) > np.random.rand():
                current_solution = new_solution
                best_profit = new_profit
                best_solution = current_solution

        # Cool down the temperature
        current_temp *= alpha

    return best_solution, best_profit

products = [Product(i) for i in range(1, 5)]
results = {}
initial_temp = 1000
final_temp = 0.1
alpha = 0.9
iterations = 100

for product in products:
    best_solution, best_profit = simulated_annealing(product, initial_temp, final_temp, alpha, iterations)
    data, avg_safety_stock = MCS_continuous(product, z_score=best_solution[0], min_order_ratio=best_solution[1])
    profit = profit_calculation(data, product)
    lost_order_proportion = len(data['units_lost']) / 365
    reorder_quantity = np.mean(data['reorder_quantities'])

    results[f'Pr{product.i}'] = {
        'best_z_score': best_solution[0],
        'best_min_order_ratio': best_solution[1],
        'mean_profit': profit,
        'lost_order_proportion': lost_order_proportion,
        'reorder_quantity': reorder_quantity,
        'average_safety_stock': avg_safety_stock
    }

for key, value in results.items():
    print(f"Product {key}:")
    print(f"  Best Z-Score: {value['best_z_score']}")
    print(f"  Best Min Order Ratio: {value['best_min_order_ratio']}")
    print(f"  Mean Profit: {value['mean_profit']}")
    print(f"  Proportion of Lost Orders: {value['lost_order_proportion']}")
    print(f"  Average Reorder Quantity: {value['reorder_quantity']}")
    print(f"  Average Safety Stock: {value['average_safety_stock']}\n")

stop = timeit.default_timer()
print('Time: ', stop - start)


Product Pr1:
  Best Z-Score: 1.2644847097606233
  Best Min Order Ratio: 0.153098579794934
  Mean Profit: 92277.82485384708
  Proportion of Lost Orders: 0.6575342465753424
  Average Reorder Quantity: 607.0805649498598
  Average Safety Stock: 9.548662974384438

Product Pr2:
  Best Z-Score: 3.0
  Best Min Order Ratio: 1.0
  Mean Profit: 457935.68609238137
  Proportion of Lost Orders: 0.8246575342465754
  Average Reorder Quantity: 3855.9369704402507
  Average Safety Stock: 22.30061285152958

Product Pr3:
  Best Z-Score: 1.9771776021540362
  Best Min Order Ratio: 0.2883861295291055
  Mean Profit: 187962.4545583188
  Proportion of Lost Orders: 0.6246575342465753
  Average Reorder Quantity: 1809.2952829494777
  Average Safety Stock: 24.29659331636644

Product Pr4:
  Best Z-Score: 1.7340550279310782
  Best Min Order Ratio: 0.26944637295909657
  Mean Profit: 382815.79994632484
  Proportion of Lost Orders: 0.18904109589041096
  Average Reorder Quantity: 406.5486473982615
  Average Safety Stock: 

## BayesianOptimization

In [10]:
start = timeit.default_timer()

def calculate_safety_stock(z, std_dev_daily_sales, lead_time):
    safety_stock = z * std_dev_daily_sales * np.sqrt(lead_time)
    return safety_stock

summary = {
    'Purchase Cost': [12, 7, 6, 37],
    'Lead Time': [9, 6, 15, 12],
    'Size': [0.57, 0.05, 0.53, 1.05],
    'Selling Price': [16.10, 8.60, 10.20, 68],
    'Starting Stock': [2750, 22500, 5200, 1400],
    'Ch': [0.2 * 12, 0.2 * 7, 0.2 * 6, 0.2 * 37],
    'Co': [1000, 1200, 1000, 1200],
    'Probability': [0.76, 1.00, 0.70, 0.23],
    'Mean Demand (Lead Time)': [103.50, 648.55, 201.68, 150.06],
    'Std. Dev. of Demand (Lead Time)': [37.32, 26.45, 31.08, 3.21],
    'Expected Demand (Lead Time)': [705, 3891, 2266, 785]
}

class Product:
    def __init__(self, i):
        self.mean = np.mean([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.sd = np.std([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.i = i
        self.unit_cost = summary['Purchase Cost'][i - 1]
        self.lead_time = summary['Lead Time'][i - 1]
        self.size = summary['Size'][i - 1]
        self.selling_price = summary['Selling Price'][i - 1]
        self.holding_cost = summary['Ch'][i - 1]
        self.ordering_cost = summary['Co'][i - 1]
        self.probability = summary['Probability'][i - 1]
        self.starting_stock = summary['Starting Stock'][i - 1]
        self.mean_demand_lead_time = summary['Mean Demand (Lead Time)'][i - 1]
        self.std_dev_demand_lead_time = summary['Std. Dev. of Demand (Lead Time)'][i - 1]
        self.expected_demand_lead_time = summary['Expected Demand (Lead Time)'][i - 1]

def daily_demand(mean, sd, probability):
    if np.random.uniform(0, 1) > probability:
        return 0
    else:
        return np.exp(np.random.normal(mean, sd))

def MCS_continuous(product, z_score=1.65, min_order_ratio=0.2):
    inventory = product.starting_stock
    mean = product.mean
    sd = product.sd
    lead_time = product.lead_time
    probability = product.probability
    demand_lead = product.expected_demand_lead_time

    high_demand_value = 2 * mean
    weights = [0.8, 0.2]
    means = [mean, high_demand_value]
    sds = [sd, sd]

    daily_sales = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
    std_dev_daily_sales = np.std(daily_sales)

    safety_stock = calculate_safety_stock(z_score, std_dev_daily_sales, lead_time)
    reorder_point = safety_stock + demand_lead

    min_order_quantity = max(int(min_order_ratio * mean), 1)

    q = 0
    stock_out = 0
    counter = 0
    order_placed = False
    data = {'inv_level': [], 'daily_demand': [], 'units_sold': [], 'units_lost': [], 'orders': [], 'reorder_quantities': []}

    total_safety_stock = 0

    for day in range(1, 365):
        day_demand = daily_demand(mean, sd, probability)
        data['daily_demand'].append(day_demand)

        if inventory <= reorder_point and not order_placed:
            q = max(reorder_point - inventory + min_order_quantity, min_order_quantity)
            order_placed = True
            data['orders'].append(q)
            data['reorder_quantities'].append(q)

        if order_placed:
            counter += 1

        if counter == lead_time:
            inventory += q
            order_placed = False
            counter = 0

        if inventory - day_demand >= 0:
            data['units_sold'].append(day_demand)
            inventory -= day_demand
        else:
            data['units_sold'].append(inventory)
            data['units_lost'].append(day_demand - inventory)
            inventory = 0
            stock_out += 1

        data['inv_level'].append(inventory)
        total_safety_stock += safety_stock

    average_safety_stock = total_safety_stock / 365

    return data, average_safety_stock

def profit_calculation(data, product):
    unit_cost = product.unit_cost
    selling_price = product.selling_price
    holding_cost = product.holding_cost
    order_cost = product.ordering_cost
    size = product.size
    days = 365

    revenue = sum(data['units_sold']) * selling_price
    Co = len(data['orders']) * order_cost
    Ch = sum(data['inv_level']) * holding_cost * size / days
    cost = sum(data['orders']) * unit_cost

    profit = revenue - cost - Co - Ch

    return profit

# Bayesian Optimization 
def bayesian_optimization(product, initial_Q, n_calls):
    dimensions = [Real(100, 10000, name='Q')]

    @use_named_args(dimensions=dimensions)
    def objective(Q):
        Q = int(Q)
        profit = np.mean([profit_calculation(MCS(Q, product)[0], product) for _ in range(10)])  # Average profit from 10 simulations
        return -profit  # Negative profit because we minimize in gp_minimize

    res = gp_minimize(objective, dimensions=dimensions, n_calls=n_calls, random_state=0)
    best_Q = int(res.x[0])
    best_profit = -res.fun

    return best_Q, best_profit

products = [Product(i) for i in range(1, 5)]
results = {}

n_calls = 100  # Number of calls to the objective function

for product in products:
    best_Q, best_profit = bayesian_optimization(product, initial_Q=10000, n_calls=n_calls)

    # best solution
    best_data, best_safety_stock = MCS(best_Q, product)
    profit_list = [profit_calculation(MCS(best_Q, product)[0], product) for _ in range(100)]
    lost_orders_list = [len(MCS(best_Q, product)[0]['units_lost']) / 365 for _ in range(100)]
    reorder_quantities = [best_Q for _ in range(100)]
    safety_stocks = [best_safety_stock for _ in range(100)]

    results[f'Pr{product.i}'] = {
        'profits': profit_list,
        'mean_profit': np.mean(profit_list),
        'std_dev_profit': np.std(profit_list),
        'lost_order_proportion': np.mean(lost_orders_list),
        'reorder_quantity': np.mean(reorder_quantities),
        'average_safety_stock': np.mean(safety_stocks)
    }


results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)

stop = timeit.default_timer()
print('Time: ', stop - start)

C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective h

                                               profits    mean_profit  \
Pr1  [52741.2201688829, 52761.483210214894, 52783.9...   52747.986598   
Pr2  [213939.249462168, 213918.13668840044, 213885....  213919.596422   
Pr3  [124674.32706732771, 124746.27143004411, 12474...  124717.822858   
Pr4  [310347.3176649884, 310392.81969677005, 310663...  310395.342436   

     std_dev_profit  lost_order_proportion  reorder_quantity  \
Pr1       37.404281             360.331068             100.0   
Pr2       36.583998             345.959808           10000.0   
Pr3       58.138751             355.082192             100.0   
Pr4      170.915294             360.535753            6076.0   

     average_safety_stock  
Pr1              7.107383  
Pr2              7.531723  
Pr3             11.251776  
Pr4              8.588879  
Time:  1282.4507155000465


In [11]:
results_df

,profits,mean_profit,std_dev_profit,lost_order_proportion,reorder_quantity,average_safety_stock
Pr1,"[52741.2201688829, 52761.483210214894, 52783.9...",52747.986598,37.404281,360.331068,100.0,7.107383
Pr2,"[213939.249462168, 213918.13668840044, 213885....",213919.596422,36.583998,345.959808,10000.0,7.531723
Pr3,"[124674.32706732771, 124746.27143004411, 12474...",124717.822858,58.138751,355.082192,100.0,11.251776
Pr4,"[310347.3176649884, 310392.81969677005, 310663...",310395.342436,170.915294,360.535753,6076.0,8.588879
